<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
#Start Here
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:

df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')

In [3]:
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [4]:
import spacy

In [5]:
from spacy.tokenizer import Tokenizer

In [6]:
nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [7]:
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(df['reviews.text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [8]:
id2word = corpora.Dictionary(df['tokens'])

In [9]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [10]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [11]:
lda.print_topics()

[(0,
  '0.046*"great" + 0.026*"easy" + 0.018*"product" + 0.017*"tablet" + 0.016*"kindle" + 0.013*"size" + 0.013*"bought" + 0.011*"price" + 0.010*"nice" + 0.009*"use"'),
 (1,
  '0.020*"kindle" + 0.018*"good" + 0.011*"great" + 0.011*"value" + 0.010*"batteries" + 0.010*"like" + 0.008*"use" + 0.008*"bought" + 0.007*"new" + 0.007*"battery"'),
 (2,
  '0.016*"use" + 0.014*"easy" + 0.012*"tablet" + 0.011*"like" + 0.011*"kindle" + 0.010*"purchased" + 0.010*"battery" + 0.009*"2" + 0.007*"books" + 0.007*"fire"'),
 (3,
  '0.037*"tablet" + 0.024*"great" + 0.015*"easy" + 0.014*"amazon" + 0.012*"good" + 0.010*"happy" + 0.009*"tablet." + 0.009*"i\'m" + 0.009*"apps" + 0.009*"fire"'),
 (4,
  '0.028*"tablet" + 0.019*"great" + 0.017*"amazon" + 0.015*"good" + 0.011*"fire" + 0.011*"use" + 0.010*"it\'s" + 0.009*"kindle" + 0.008*"apps" + 0.008*"it."'),
 (5,
  '0.035*"love" + 0.018*"like" + 0.013*"kindle" + 0.013*"read" + 0.013*"easy" + 0.012*"great" + 0.011*"it\'s" + 0.011*"it." + 0.010*"screen" + 0.008*"litt

In [12]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [13]:
topics = [' '.join(t[0:5]) for t in words]

In [14]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
great easy product tablet kindle

------ Topic 1 ------
kindle good great value batteries

------ Topic 2 ------
use easy tablet like kindle

------ Topic 3 ------
tablet great easy amazon good

------ Topic 4 ------
tablet great amazon good fire

------ Topic 5 ------
love like kindle read easy

------ Topic 6 ------
great tablet love use it's

------ Topic 7 ------
kids great tablet games loves

------ Topic 8 ------
batteries great batteries. brand amazon

------ Topic 9 ------
loves bought it. old year

------ Topic 10 ------
year fire old tablet love

------ Topic 11 ------
great amazon tablet price. good

------ Topic 12 ------
tablet love great works amazon

------ Topic 13 ------
great use kindle fire it's

------ Topic 14 ------
batteries good great work long



In [15]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [16]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/huanqingxu/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.258995  0.034943       1        1  14.432901
8     -0.216977  0.054843       2        1  10.837570
9      0.118092  0.160867       3        1  10.191772
5      0.028585 -0.070204       4        1   6.854102
10     0.109271  0.043767       5        1   6.654129
4      0.041711 -0.044686       6        1   6.424335
7      0.100448  0.075017       7        1   6.221262
3      0.033287 -0.104569       8        1   5.718868
0      0.039496 -0.009935       9        1   5.587534
11     0.008720 -0.061478      10        1   5.006360
12     0.030818 -0.051014      11        1   4.833097
13     0.014029 -0.087070      12        1   4.776938
6      0.032103  0.002917      13        1   4.432137
1     -0.116972  0.009420      14        1   4.373461
2      0.036385  0.047182      15        1   3.655536, topic_info=      Category         Freq        Term        Total  loglift  logprob
20     Default  5079.000000   batteries  5079.000000  30.0000  30.0000
4459   Default  2163.000000       loves  2163.000000  29.0000  29.0000
7784   Default  4749.000000      tablet  4749.000000  28.0000  28.0000
126    Default  1556.000000        year  1556.000000  27.0000  27.0000
141    Default  1388.000000  batteries.  1388.000000  26.0000  26.0000
1411   Default  1709.000000         old  1709.000000  25.0000  25.0000
78     Default  7629.000000       great  7629.000000  24.0000  24.0000
109    Default  3332.000000        love  3332.000000  23.0000  23.0000
5566   Default  2303.000000      kindle  2303.000000  22.0000  22.0000
1338   Default  2763.000000        easy  2763.000000  21.0000  21.0000
32     Default  3000.000000      bought  3000.000000  20.0000  20.0000
67     Default  4871.000000        good  4871.000000  19.0000  19.0000
22     Default   959.000000       brand   959.000000  18.0000  18.0000
660    Default  2453.000000         it.  2453.000000  17.0000  17.0000
26     Default  2038.000000        long  2038.000000  16.0000  16.0000
1405   Default  1833.000000        fire  1833.000000  15.0000  15.0000
23     Default  1852.000000        work  1852.000000  14.0000  14.0000
239    Default  1912.000000        kids  1912.000000  13.0000  13.0000
2555   Default   727.000000    daughter   727.000000  12.0000  12.0000
1130   Default  1128.000000       games  1128.000000  11.0000  11.0000
28     Default  3126.000000      amazon  3126.000000  10.0000  10.0000
3312   Default  1267.000000        apps  1267.000000   9.0000   9.0000
14     Default  2275.000000        like  2275.000000   8.0000   8.0000
1594   Default   783.000000        gift   783.000000   7.0000   7.0000
114    Default  3074.000000         use  3074.000000   6.0000   6.0000
665    Default  1072.000000        play  1072.000000   5.0000   5.0000
1393   Default   980.000000        read   980.000000   4.0000   4.0000
160    Default   735.000000        size   735.000000   3.0000   3.0000
19     Default  2663.000000       price  2663.000000   2.0000   2.0000
5604   Default   893.000000       books   893.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
5566   Topic15   136.880524      kindle  2303.005371   0.4861  -4.5462
3034   Topic15    26.910343      laptop   117.955963   1.8311  -6.1728
2456   Topic15    28.051399   grandkids   128.058685   1.7905  -6.1313
12992  Topic15    56.821514     tablets   500.183350   1.1339  -5.4254
1386   Topic15    26.369524        ease   122.501640   1.7730  -6.1931
4883   Topic15    24.108982        tech   104.209244   1.8451  -6.2827
331    Topic15    69.049286         new   893.546143   0.7486  -5.2305
7784   Topic15   153.090012      tablet  4749.605957  -0.1259  -4.4343
14199  Topic15    46.407413    parental   404.935150   1.1427  -5.6279
643    Topic15    35.313419     storage   231.029205   1.4306  -5.9010
1405   Topic15    85.534569        fire  1833.175659   0.

In [17]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [19]:
df = pd.DataFrame.from_records(new_distro)
df.columns = topics

In [20]:
df.head()

,great easy product tablet kindle,kindle good great value batteries,use easy tablet like kindle,tablet great easy amazon good,tablet great amazon good fire,love like kindle read easy,great tablet love use it's,kids great tablet games loves,batteries great batteries. brand amazon,loves bought it. old year,year fire old tablet love,great amazon tablet price. good,tablet love great works amazon,great use kindle fire it's,batteries good great work long
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.316669,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.616382
1,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.011112,0.844434
2,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.766622
3,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844442
4,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.844442


## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling